# Primitives examples

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: -0.14274631497284718
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [IQP(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: 0.3711340206185567
>>> Standard errors for PUB 0: 0.16935809977687646
>>> Expectation values for PUB 1: -0.026905829596412557
>>> Standard errors for PUB 1: 0.21577124738663342
>>> Expectation values for PUB 2: 0.005939123979213066
>>> Standard errors for PUB 2: 0.2376372001854862


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 0.96674749  0.91218758  0.7620351   0.55371545  0.26626138 -0.02299633
  -0.31112676 -0.5864063  -0.80396958 -0.93563482 -0.96562022 -0.91015849
  -0.76857327 -0.57671177 -0.27031955  0.03426903  0.31495948  0.58708266
   0.78142416  0.92616574  0.97080567]
 [ 0.0324654   0.33885762  0.58234813  0.79111869  0.91511849  0.96043477
   0.91173667  0.78029689  0.54988273  0.30188314 -0.01239998 -0.32871218
  -0.57783904 -0.78841324 -0.92233302 -0.97463839 -0.91444212 -0.76293692
  -0.55033363 -0.27911227  0.02502541]
 [-0.03021086 -0.31743947 -0.59745356 -0.81231138 -0.94442753 -0.98681291
  -0.92639119 -0.78570779 -0.55777362 -0.29376679  0.02389814  0.3054904
   0.58618085  0.79585323  0.94082026  0.99087109  0.9239112   0.7836787
   0.54898091  0.31563584 -0.03517085]
 [ 0.9834311   0.92436211  0.78322779  0.56701724  0.28835589 -0.01803633
  -0.32578128 -0.57964268 -0.81253684 -0.93743845 -0.97869656 -0.92796937
  -0.78773688 -0.5618318  -0.27279955  0.022094

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

<CloudContent>
<Admonition type="caution">
The following code block will return an error for users on the Open Plan, because it uses sessions. Workloads on the Open Plan can run only in [job mode](/guides/execution-modes#job-mode) or [batch mode](/guides/execution-modes#batch-mode).
</Admonition>
</CloudContent>

In [4]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: -0.12389380530973451
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: 0.17486338797814208
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['1111011100111110101011110111010000101000000000111001100001000100110010110011000010100111001000010001000010100110000110010000001', '0001001010001101100100010100000010001000000100011000001110011101100010010011110101111100111001100010000001001101110010000110101', '1001101011001010001010001000100001000100010011000000100101110010111010010010001001010001000011011010100000001100100110101010001', '0111101000001100010100100110111100011000010001001000111111100010000010010000000001001110001110100010110011001101100000000000101', '0010000001011010000101110101101000001110110100000001100010111110011000010111000010000100011110001001001100000001010011100111101', '0000000111100100011000010011001010000000010111100101000111000000101000111000000011100100000001001101010111000100000000010100100', '0101011101011100101011001110100110001010010110110111111101000000001010000011000011001011011110010011000110101010010101100110101', '10010010010000000110001110110010001101100001001100001000010

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [IQP(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['1101010010001101110000010100000110111011000001011010010000001001101001010011011000011001100110100101110000010000000011000111101', '1111111100001000110101110011100010101100100111000001000010110111111000001000010010000110111011000101101010110001101000110010100', '0010101111001100111101001001010110000111101011000100010000011001000011011100001010001001001110101111010101000010000110110010000', '0011010001001101010110111101000001111101101001001110000000111000011010000100011010001001001111100111110000000000100010111010100', '0011110010101110101001010000010110111111100100100001011011110101110000011100011111100011010011110011101000010100100010110110100', '0100011010100100111100011111010011001100001110000010010011011010110010010110001000100000001010001001111100111100110000010000011', '1111100111101010110001011001100010000000100010011011010010010001000001100001010100111101110100100111001111100000000010000001010', '0010110100000011100100110100100000100011011111110

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import RealAmplitudes
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = RealAmplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['0100101011011110111011110111111101011110010110010010011000111111010000001001001100011110000111111100100010000110001010101110101', '0001110111100101010000001101110011000011110000011011100000011111011100010110101110100010111110011011010101101000111010100010101', '0100110001001101110001000111101100100001010010101000100011010001100000010101010111001000101100001001010101110000110111100010010', '1001010001111001011010001110110011000010101111000111000010011011000100101001011001001011000111010011011000111110101110100011110', '0001011010010010010001100101111111100101100110011010001010111011101010110000101001110001011010101101001100100101010101101111101', '0010111000010000100001000101011110010011000111010101101001001010010100010100111010011000110000001011100100011101011001011100101', '0000101001101110100111001111010111101110101111100011101110100111100110001011111011111100100001111111111100100001001101100110001', '00011010110001110110110011011

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

<CloudContent>
<Admonition type="caution">
The following code block will return an error for users on the Open Plan, because it uses sessions. Workloads on the Open Plan can run only in [job mode](/guides/execution-modes#job-mode) or [batch mode](/guides/execution-modes#batch-mode).
</Admonition>
</CloudContent>

In [8]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['1101101110011111000110011110100110111001011010000011001001100100111101101010010100000000010110000001010000000000000000100111110', '0100011110111011001110110110100010110000111100100100100100100011010000111001110111000001011111100101010100001100110000101010111', '1001011010001000001111010011000101000001001110100110100011000010110100000111010001101010010110100010010110010110101101000010100', '0001010010000000011000111000100011110000101110001111011000100000110110101010110100000000000000101001001100011010011100100010110', '1111110010010101001001100110110000100110111101000000000100110100010010111111000101100011000101101001001110000100101001101101100', '0100001101110000001000010010000111110110111100101000010000100111111101010100100001110010100011000000011000010100001000100010110', '1101011111001011000110010111110100000000010011000001010010010110110010100110110100000010001100010001001000100100100101101000100', '1000010111000010011011101110110001

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['1100110001101000010000010110001000001000111100000100110000000000110100011000100100001100000000011111010010100100100000100000011', '0001001000100001000101101100011110101011010110000000110000000100100101111001111010100000000001001000010110010111100000100000001', '0111000000110000101110101110000101000001001011000100100000000001101100010011110000101110100010010011010010110100111000010000010', '1010101010110101001010000000101000101011000101110101000110110010110010010110100000000110111011111010010100011111110010110000010', '0100101000101001100011011100000010001011100010100110110110100010000010010000001100100010010000110100000101101010000010100000101', '1101110001100011100100011110010100000101001000101000010101010000010111001000011100000111000100010110010111001001100001000000011', '0101001001100101001111100010001101000000011101010101111000000000000000101010011001100010100011100111001101100111100001000100010', '1101110001110011110101000101111100

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Submit pre-transpiled circuits](https://learning.quantum.ibm.com/tutorial/submitting-user-transpiled-circuits-using-primitives) tutorial.
    - Read [Migrate to V2 primitives](/migration-guides/v2-primitives).
    - Understand the [Job limits](/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>